### tf 1.x  >>  tf 2.x + keras 실험 코드

In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Conv2D
tf.__version__

'2.1.0'

In [2]:
# input image setting
img_shape = [512, 680, 3]
img_height = img_shape[0]
img_width = img_shape[1]

In [3]:
######### 예제 input img 생성 + 전처리 #########
# size = 1 , 512, 680, 3 batch,height,width,channel
ximg = cv2.imread('data/1/1/1_1_000.png')
ximg = ximg.reshape(1,512,680,3)
# 픽셀값 예시 print
print(ximg[0][0:3,0:1,0:1])
ximg = ximg / 127.5 - 1. # -1 ~ 1 의 scale로 변환
print(ximg[0][0:3,0:1,0:1])

[[[229]]

 [[228]]

 [[226]]]
[[[0.79607843]]

 [[0.78823529]]

 [[0.77254902]]]


In [4]:
########### 마스크 생성 ############
maxtop = img_height - 128
maxleft = img_width - 128
# maxtop = 0 ~ (512-128)까지의 랜덤 텐서값 하나 생성
t = tf.keras.backend.random_uniform(
    [], minval=0, maxval=maxtop, dtype=tf.int32)
# maxleft = 0 ~ (680-128)까지의 랜덤 텐서값 하나 생성
l = tf.keras.backend.random_uniform(
    [], minval=0, maxval=maxleft, dtype=tf.int32)
h = tf.constant(img_shape[0])
w = tf.constant(img_shape[1])

# bbox ex)329,212,512,512 / x=329, y=212 점의 512*512 사각형
bbox = (t,l,h,w)
print(bbox)

(<tf.Tensor: shape=(), dtype=int32, numpy=275>, <tf.Tensor: shape=(), dtype=int32, numpy=236>, <tf.Tensor: shape=(), dtype=int32, numpy=512>, <tf.Tensor: shape=(), dtype=int32, numpy=680>)


In [5]:
def npmask(bbox, height, width, delta_h, delta_w):
    mask = np.zeros((1, height, width, 1), np.float32)
    h = np.random.randint(delta_h // 2 + 1)
    w = np.random.randint(delta_w // 2 + 1)
    mask[:, bbox[0] + h:bbox[0] + bbox[2] - h,
    bbox[1] + w:bbox[1] + bbox[3] - w, :] = 1.
    print(bbox[0] + h,':',bbox[0] + bbox[2] - h)
    print(bbox[1] + w,':',bbox[1] + bbox[3] - w)
    return mask
# 32 = max_delta_height
# npmask ex) [:, 339:831, 223:881, :] = 1. / 일정범위 1값으로 채우기
mask = tf.py_function(npmask,[bbox,img_height,img_width,32,32],tf.float32)
mask.set_shape([1] + [img_height, img_width] + [1])

tf.Tensor(276, shape=(), dtype=int32) : tf.Tensor(786, shape=(), dtype=int32)
tf.Tensor(237, shape=(), dtype=int32) : tf.Tensor(915, shape=(), dtype=int32)


In [6]:
# 흰 부분이 1이니깐 검<->흰 변경
# 흰픽셀이 마스킹될 부분이다.
mask = (1. - mask) # 최종적으로 모델에 들어갈 마스크
ximg = ximg * mask
x_copy = ximg
ones_x = tf.ones_like(ximg)[:, :, :, 0:1]
x_input = tf.concat([ximg,ones_x,mask],axis=3)
x_input

<tf.Tensor: shape=(1, 512, 680, 5), dtype=float32, numpy=
array([[[[ 0.79607844,  0.7490196 ,  0.69411767,  1.        ,
           1.        ],
         [ 0.7882353 ,  0.7411765 ,  0.6862745 ,  1.        ,
           1.        ],
         [ 0.78039217,  0.73333335,  0.6784314 ,  1.        ,
           1.        ],
         ...,
         [ 0.8745098 ,  0.8745098 ,  0.8745098 ,  1.        ,
           1.        ],
         [ 0.8745098 ,  0.8745098 ,  0.8745098 ,  1.        ,
           1.        ],
         [ 0.8745098 ,  0.8745098 ,  0.8745098 ,  1.        ,
           1.        ]],

        [[ 0.7882353 ,  0.7411765 ,  0.6862745 ,  1.        ,
           1.        ],
         [ 0.78039217,  0.73333335,  0.6784314 ,  1.        ,
           1.        ],
         [ 0.77254903,  0.7254902 ,  0.67058825,  1.        ,
           1.        ],
         ...,
         [ 0.8745098 ,  0.8745098 ,  0.8745098 ,  1.        ,
           1.        ],
         [ 0.8745098 ,  0.8745098 ,  0.8745098 ,  1.

## Model Start

In [7]:
# variable_scope 안써도됩니다.
# arg_scope 인자들을 일제히 설정해주는 코드. 무시 가능
'''
tf.get_variable 대신에 tf.Variable을 사용하세요.
모든 variable_scope는 파이썬 객체로 바꿀 수 있습니다. 일반적으로 다음 중 하나가 될 것입니다:
-tf.keras.layers.Layer
-tf.keras.Model
-tf.Module
만약 (tf.Graph.get_collection(tf.GraphKeys.VARIABLES)처럼) 
변수의 리스트가 필요하다면 Layer와 Model 객체의 .variables이나 .trainable_variables 속성을 사용하세요.
'''
# tf.function 데코레이터는 선택 사항으로 성능을 위해 추가할 수 있습니다. (알아보기)

# 주의깊게 볼 사항
# padding = SYMETRIC , SAME , REFELECT ?
# input image (none, height, width, 3+2) ?
# dilation_rate / conv 인자중 하나 무슨의미 ?
# context attention

'\ntf.get_variable 대신에 tf.Variable을 사용하세요.\n모든 variable_scope는 파이썬 객체로 바꿀 수 있습니다. 일반적으로 다음 중 하나가 될 것입니다:\n-tf.keras.layers.Layer\n-tf.keras.Model\n-tf.Module\n만약 (tf.Graph.get_collection(tf.GraphKeys.VARIABLES)처럼) \n변수의 리스트가 필요하다면 Layer와 Model 객체의 .variables이나 .trainable_variables 속성을 사용하세요.\n'

In [8]:
def gated_conv(x):
    x,y = tf.split(x,2,3)
    x = tf.nn.elu(x)
    y = tf.nn.sigmoid(y)
    return x*y

input = Input(shape=(512,680,5))
# cnum = 48
# filter=48, ksize=5, stride=1
x = Conv2D(48,5,1,dilation_rate=1,padding='SAME')(input)
x = gated_conv(x)

# ksize =2 (downsampling)
x1 = Conv2D(96,3,2,dilation_rate=1,padding='SAME')(x)
x1 = gated_conv(x1)

x2 = Conv2D(96,3,1,dilation_rate=1,padding='SAME')(x1)
x2 = gated_conv(x2)

# ksize =2 (downsampling)
x3 = Conv2D(192,3,2,dilation_rate=1,padding='SAME')(x2)
x3 = gated_conv(x3)

#conv5
x4 = Conv2D(192,3,1,dilation_rate=1,padding='SAME')(x3)
x4 = gated_conv(x4)

#conv6
x5 = Conv2D(192,3,1,dilation_rate=1,padding='SAME')(x4)
x5 = gated_conv(x5)

# resize mask
x5_shape = x5.get_shape().as_list()[1:3]
mask_s = tf.image.resize(
    mask, [x5_shape[0],x5_shape[1]], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR
)
print('mask origin',mask.shape)
print('mask reshape',mask_s.shape)

model = Model(input,x5)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mse'])
model.summary()

mask origin (1, 512, 680, 1)
mask reshape (1, 128, 170, 1)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 680, 5) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 680, 48) 6048        input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_split (TensorFlowOp [(None, 512, 680, 24 0           conv2d[0][0]                     
__________________________________________________________________________________________________
tf_op_layer_Elu (TensorFlowOpLa [(None, 512, 680, 24 0           tf_op_layer_split[0][0]          
___________________________________